# Generate FAQ for README

Let's get start with pandas-chatflow!!

In [1]:
from openai import OpenAI
from openaivec import pandas_ext
from pydantic import BaseModel, Field
import pandas as pd

pandas_ext.use(OpenAI())

pandas_ext.responses_model("gpt-4.1")

The model name 'gpt-4.1' is not supported by tiktoken. Instead, using the 'o200k_base' encoding.


In [2]:
docs_df: pd.DataFrame = pd.DataFrame(
    {"body": [open("../README.md").read()]}
)
docs_df

,body
0,# What is this?\n\nLet's dive into **Generativ...


In [3]:
class Question(BaseModel):
    question: str = Field(description="The question to ask the model.")
    answer: str = Field(description="The answer to the question.")

class Section(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")
    questions: list[Question] = Field(description="List of questions and answers related to the section.")

class Document(BaseModel):
    sections: list[Section] = Field(description="List of sections in the document.")

In [4]:
sections_df = docs_df.pipe(
    lambda df: df
    .assign(
        section=lambda df: df["body"].ai.responses(
            instructions="""
            Generate a list of FAQ for each section of the document. 
            Break down the document into as many detailed sections as possible, 
            regardless of their order and markdown format.
            """,
            response_format=Document
        )
        .map(lambda x: x.sections)
    )
    .drop(columns=["body"])
    .explode("section")
    .ai.extract("section")
)

sections_df


,section_title,section_content,section_questions
0,Introduction to Generative Mutation for Tabula...,This section introduces the concept of generat...,[{'question': 'What is generative mutation for...
0,Overview of openaivec Package,This section provides an overview of the opena...,[{'question': 'What is the main purpose of the...
0,Installation and Basic Usage,This section explains how to install and unins...,"[{'question': 'How do you install openaivec?',..."
0,Using openaivec with Pandas DataFrame,This section describes how to extend pandas.Se...,[{'question': 'How do you enable openaivec ext...
0,Using openaivec with Apache Spark UDF,This section explains how to create and regist...,[{'question': 'How do you create a UDF for Spa...
0,Building Prompts with FewShotPromptBuilder,This section covers how to build effective pro...,[{'question': 'What is few-shot learning in th...
0,Using openaivec with Microsoft Fabric,This section provides step-by-step instruction...,[{'question': 'How do you set up openaivec in ...
0,Contributing to openaivec,This section outlines the guidelines for contr...,[{'question': 'How can you contribute to opena...
0,Community and Support,This section provides information about the op...,[{'question': 'How can you join the openaivec ...


In [5]:
questions_df: pd.DataFrame = sections_df.pipe(
    lambda df: df
    .drop(columns=["section_content"])
    .explode("section_questions")
    .ai.extract("section_questions")
    .reset_index(drop=True)
)


In [6]:
from IPython.display import Markdown, display

display(Markdown(questions_df.to_markdown()))

|    | section_title                                        | section_questions_question                                                   | section_questions_answer                                                                                                                                                                                                          |
|---:|:-----------------------------------------------------|:-----------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Introduction to Generative Mutation for Tabular Data | What is generative mutation for tabular data?                                | Generative mutation refers to the process of transforming or mutating tabular data (such as pandas Series or DataFrames) using natural language instructions, often powered by large language models (LLMs) like OpenAI's models. |
|  1 | Introduction to Generative Mutation for Tabular Data | How do you mutate a pandas Series with natural language?                     | You can use the ai.responses accessor on a pandas Series to apply natural language instructions. For example: animals.ai.responses('Translate the animal names to Chinese.').                                                     |
|  2 | Introduction to Generative Mutation for Tabular Data | What is the result of translating animal names to Chinese using this method? | The result is a list of Chinese translations for the animal names, e.g., ['熊猫', '考拉', '蟒蛇', '狗', '猫'].                                                                                                                    |
|  3 | Introduction to Generative Mutation for Tabular Data | Can you perform multiple mutations at once?                                  | Yes, you can use the assign method in pandas to add multiple mutated columns at once, each with its own natural language instruction.                                                                                             |
|  4 | Overview of openaivec Package                        | What is the main purpose of the openaivec package?                           | The main purpose is to provide a vectorized interface for the OpenAI API, enabling efficient processing of multiple inputs at once and seamless integration with data processing tools like Pandas and Apache Spark.              |
|  5 | Overview of openaivec Package                        | What are the key features of openaivec?                                      | Key features include vectorized API requests, integration with Pandas DataFrames, a UDF builder for Apache Spark, and compatibility with multiple OpenAI clients including Azure OpenAI.                                          |
|  6 | Overview of openaivec Package                        | What are the requirements to use openaivec?                                  | You need Python 3.10 or higher.                                                                                                                                                                                                   |
|  7 | Installation and Basic Usage                         | How do you install openaivec?                                                | Run pip install openaivec in your terminal.                                                                                                                                                                                       |
|  8 | Installation and Basic Usage                         | How do you uninstall openaivec?                                              | Run pip uninstall openaivec in your terminal.                                                                                                                                                                                     |
|  9 | Installation and Basic Usage                         | How do you use openaivec with the OpenAI client?                             | You initialize a VectorizedResponsesOpenAI client with your desired parameters and use the parse method to process a list of inputs.                                                                                              |
| 10 | Using openaivec with Pandas DataFrame                | How do you enable openaivec extensions in pandas?                            | Import pandas_ext from openaivec and call pandas_ext.use(OpenAI()) to set the OpenAI client.                                                                                                                                      |
| 11 | Using openaivec with Pandas DataFrame                | How do you specify models for responses and embeddings?                      | Use pandas_ext.responses_model('model-name') and pandas_ext.embeddings_model('model-name') to set the models.                                                                                                                     |
| 12 | Using openaivec with Pandas DataFrame                | How do you apply natural language mutations to a DataFrame column?           | Use the assign method with lambda functions that call ai.responses on the desired Series.                                                                                                                                         |
| 13 | Using openaivec with Apache Spark UDF                | How do you create a UDF for Spark with openaivec?                            | Use UDFBuilder.of_azureopenai or UDFBuilder with the appropriate parameters to create a UDF, then register it with spark.udf.register.                                                                                            |
| 14 | Using openaivec with Apache Spark UDF                | What are some example use cases for Spark UDFs with openaivec?               | Examples include extracting flavor or product type from product names using natural language instructions.                                                                                                                        |
| 15 | Using openaivec with Apache Spark UDF                | How do you use the registered UDFs in Spark SQL?                             | Call the UDFs in your SQL queries, e.g., SELECT id, product_name, parse_taste(product_name) AS taste FROM product_names;                                                                                                          |
| 16 | Building Prompts with FewShotPromptBuilder           | What is few-shot learning in the context of LLMs?                            | Few-shot learning involves providing a few examples in the prompt to guide the LLM's behavior and improve its performance.                                                                                                        |
| 17 | Building Prompts with FewShotPromptBuilder           | How do you build a few-shot prompt with openaivec?                           | Use FewShotPromptBuilder to set the purpose, cautions, and examples, then call build to generate the prompt.                                                                                                                      |
| 18 | Building Prompts with FewShotPromptBuilder           | What does the improve method do?                                             | The improve method uses OpenAI's API to refine the prompt by eliminating contradictions, ambiguities, and redundancies, iterating up to a specified number of times.                                                              |
| 19 | Using openaivec with Microsoft Fabric                | How do you set up openaivec in Microsoft Fabric?                             | Create a new environment in Microsoft Fabric, add openaivec from PyPI to the environment's custom library, and select the environment in your notebook.                                                                           |
| 20 | Using openaivec with Microsoft Fabric                | How do you use openaivec in a Microsoft Fabric notebook?                     | Import and use openaivec.spark.UDFBuilder as you would in a regular Python environment.                                                                                                                                           |
| 21 | Contributing to openaivec                            | How can you contribute to openaivec?                                         | Fork the repository, create a branch from main, add tests for new code, ensure tests pass, and make sure your code lints.                                                                                                         |
| 22 | Contributing to openaivec                            | How do you install development dependencies?                                 | Run uv sync --all-extras --dev.                                                                                                                                                                                                   |
| 23 | Contributing to openaivec                            | How do you reformat the code?                                                | Run uv run ruff check . --fix.                                                                                                                                                                                                    |
| 24 | Community and Support                                | How can you join the openaivec community?                                    | Join the Discord community for developers at https://discord.gg/vbb83Pgn.                                                                                                                                                         |